In [157]:
import pandas as pd
import numpy as np

In [158]:
# Load data
county_ba = pd.read_csv('../../../data/county_to_ba.csv')
ba_to_state = pd.read_csv('../../../data/ba_to_state.csv')
aeo_2023 = pd.read_csv('../../../data/demand_AEO_2023_reference.csv')
county_map = pd.read_csv('../../../data/dgen_county_fips_mapping.csv')
load_growth = pd.read_csv('../../../data/load_growth_to_model.csv')

In [159]:
# First extract unique combinations from balance area to state mapping
ba_to_state_unique = ba_to_state[['ba', 'state']].drop_duplicates(subset=['ba', 'state'])

In [160]:
# Re-base AEO 2023 data to 2026

# Get the 2026 multiplier per state
base_multipliers = aeo_2023[aeo_2023['year'] == 2026][['state', 'multiplier']].rename(columns={'multiplier': 'base_multiplier'})

# Merge with the original DataFrame
aeo_2023 = aeo_2023.merge(base_multipliers, on='state', how='left')

# Rebase the multiplier
aeo_2023['multiplier'] = aeo_2023['multiplier'] / aeo_2023['base_multiplier']

# Only years after 2025
aeo_2023 = aeo_2023[aeo_2023['year'] > 2025]

# Join load growth data with ba_to_state_unique to get ba associated with load growth multipliers
aeo_2023_ba = aeo_2023.merge(ba_to_state_unique, on='state', how='inner')

# Add sector
aeo_2023_ba['sector_abbr'] = 'res'
aeo_2023_ba['load_growth_scenario_2023'] = 'AEO2023 Reference'


In [161]:
# Join aeo data with county_ba mapping to get load growth multipliers for each county
aeo_2023_county = aeo_2023_ba.merge(county_ba, on='ba', how='inner')

In [162]:
# Join aeo data to NERC regions and format eventual df for export
load_growth = (
    load_growth[['year', 'sector_abbr', 'county_id', 'nerc_region_desc', 'nerc_region_abbr', 'load_multiplier', 'load_growth_scenario']][load_growth['year'] > 2025]
    .merge(aeo_2023_county[['county_id', 'year', 'sector_abbr', 'multiplier', 'load_growth_scenario_2023']], on=['county_id', 'year', 'sector_abbr'], how='left')
)

In [163]:
# Substitute 2023-based load multipliers for residential sector
load_growth['load_multiplier'] = np.where(load_growth['sector_abbr'] == 'res', load_growth['multiplier'], load_growth['load_multiplier'])
load_growth['load_growth_scenario'] = np.where(load_growth['sector_abbr'] == 'res', load_growth['load_growth_scenario_2023'], load_growth['load_growth_scenario'])

# Ensure dtypes are correct
load_growth['load_multiplier'] = load_growth['load_multiplier'].astype(float)

In [164]:
# Write the load growth file
load_growth[['year', 'sector_abbr', 'county_id', 'nerc_region_desc', 'nerc_region_abbr', 'load_growth_scenario', 'load_multiplier']].to_csv('../../../data/load_growth_to_model_adjusted.csv', index = False)